In [14]:
import pandas as pd
import sys
sys.path.append('../')
from word_identification import WordIdentifier

In [3]:
chem_tokenizer = Tokenizer.from_file('../data/vocab.json')

In [4]:
from pathlib import Path 
bdb = pd.read_csv('../data/bdb/interactions.csv')

In [17]:
chem_tokenizer = WordIdentifier.from_file('../data/vocab.json')

In [35]:
doc_to_tokens = {}
vocab = set()
for row in bdb.iterrows():
    doc_id, smiles = row[1]['prot_id'], row[1]['smiles']
    smiles_token_ids = sum(chem_tokenizer.identify_words([smiles], out_type='int'), [])
    doc_to_tokens[doc_id] = ' '.join([str(number) for number in smiles_token_ids])
    vocab |= set(smiles_token_ids)

vocab = {str(number) for number in vocab}
corpus = list(doc_to_tokens.values())

In [40]:
len(vocab)

1601

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
count_model = CountVectorizer(ngram_range=(1,1), token_pattern=r'(?u)\b\w+\b', vocabulary=vocab) # default unigram model 
X = count_model.fit_transform(corpus)
# X[X > 0] = 1 # run this line if you don't want extra within-text cooccurence (see below)
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0
print(Xc.todense()) # print out matrix in dense format

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


C:\Users\gokce\Anaconda3\envs\transformers\lib\site-packages\scipy\sparse\_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [48]:
len(count_model.vocabulary_)

1601

In [49]:
print(Xc.shape)

(1601, 1601)


In [62]:
import scipy
cx = scipy.sparse.coo_matrix(Xc)
cooccurrences = [(chem_tokenizer.tokenizer.id_to_token(i),chem_tokenizer.tokenizer.id_to_token(j),v) for i,j, v in zip(cx.row, cx.col, cx.data) if v != 0]
df_cooccurrence = pd.DataFrame.from_records(cooccurrences, columns=['word1', 'word2', 'count'])

In [64]:
df_cooccurrence.to_csv('../data/bdb/cooccurrence.csv', index=False)